# Explicabilidad con SHAP
## Análisis de Interpretabilidad para Modelos AML

**Objetivo Específico 4 (OE4)**: Evaluar el desempeño de los modelos desarrollados mediante métricas de precisión, recall, F1-score y análisis de explicabilidad.

Este notebook implementa análisis de explicabilidad usando **SHAP** (SHapley Additive exPlanations) para:
- Identificar qué features influyen más en las predicciones de fraude
- Entender decisiones individuales del modelo
- Validar que el modelo aprende patrones sensatos
- Cumplir con requisitos de transparencia regulatoria (SARLAFT 2.0)

### ¿Por qué SHAP?

SHAP es esencial para:
1. **Regulación**: SARLAFT 2.0 requiere explicabilidad en decisiones AML
2. **Confianza**: Entidades financieras necesitan justificar alertas
3. **Debug**: Detectar si el modelo aprende patrones erróneos
4. **Mejora**: Identificar features relevantes para ingeniería

In [ ]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Utilidades del proyecto
from src.utils.config import load_config
from src.utils.reproducibility import set_seed

# Configuración
config = load_config('../../configs/config.yaml')
set_seed(config['project']['random_seed'])

# Estilo de visualización
plt.style.use('seaborn-v0_8-whitegrid')
shap.initjs()  # Inicializa JS para visualizaciones interactivas

print("✓ Librerías cargadas")
print(f"  SHAP version: {shap.__version__}")

## 1. Carga y Preprocesamiento de Datos

In [ ]:
# Carga datos
df = pd.read_csv('../../data/synthetic/aml_colombia_synthetic.csv')
print(f"📊 Dataset: {df.shape}")
print(f"   Fraude: {df['isFraud'].sum()} ({df['isFraud'].mean()*100:.2f}%)")

# Preprocesamiento
le = LabelEncoder()
df['type_encoded'] = le.fit_transform(df['type'])

# Features
feature_names = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
                'oldbalanceDest', 'newbalanceDest', 'type_encoded']

X = df[feature_names].values
y = df['isFraud'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=config['model']['test_size'],
    random_state=config['model']['random_state'],
    stratify=y
)

# Scale (SHAP funciona mejor con datos escalados)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ Train: {X_train.shape}, Test: {X_test.shape}")
print(f"  Features: {feature_names}")

## 2. Entrenar Modelos para Explicabilidad

Entrenamos XGBoost y Random Forest para analizar con SHAP.

In [ ]:
# Calcula weight para balanceo
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}")

# XGBoost
print("\n🔥 Entrenando XGBoost...")
xgb_params = config['xgboost']
model_xgb = xgb.XGBClassifier(
    n_estimators=xgb_params['n_estimators'],
    max_depth=xgb_params['max_depth'],
    learning_rate=xgb_params['learning_rate'],
    subsample=xgb_params['subsample'],
    colsample_bytree=xgb_params['colsample_bytree'],
    scale_pos_weight=scale_pos_weight,
    random_state=config['model']['random_state'],
    tree_method='hist'
)
model_xgb.fit(X_train_scaled, y_train)
print("✓ XGBoost entrenado")

# Random Forest
print("\n🌳 Entrenando Random Forest...")
rf_params = config['random_forest']
model_rf = RandomForestClassifier(
    n_estimators=rf_params['n_estimators'],
    max_depth=rf_params['max_depth'],
    min_samples_split=rf_params['min_samples_split'],
    min_samples_leaf=rf_params['min_samples_leaf'],
    class_weight=rf_params['class_weight'],
    random_state=config['model']['random_state'],
    n_jobs=-1
)
model_rf.fit(X_train_scaled, y_train)
print("✓ Random Forest entrenado")

## 3. SHAP para XGBoost

In [ ]:
# Crea explainer para XGBoost
print("📊 Calculando SHAP values para XGBoost...")
explainer_xgb = shap.TreeExplainer(model_xgb)

# Calcula SHAP values (usa subset para eficiencia)
sample_size = min(1000, len(X_test_scaled))
X_test_sample = X_test_scaled[:sample_size]
shap_values_xgb = explainer_xgb.shap_values(X_test_sample)

print(f"✓ SHAP values calculados para {sample_size} muestras")
print(f"  Shape: {shap_values_xgb.shape}")

### 3.1 Summary Plot - Feature Importance Global

In [ ]:
# Summary plot (importancia global)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values_xgb, X_test_sample, 
                 feature_names=feature_names, show=False)
plt.title('SHAP Summary Plot - XGBoost', fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../../reports/figures/shap_summary_xgb.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Interpretación del Summary Plot:")
print("  - Features ordenadas por importancia (top = más importante)")
print("  - Color rojo = valor alto de la feature")
print("  - Color azul = valor bajo de la feature")
print("  - Eje X = impacto en predicción (positivo = aumenta prob. fraude)")

### 3.2 Bar Plot - Feature Importance Promedio

In [ ]:
# Bar plot (importancia promedio)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values_xgb, X_test_sample,
                 feature_names=feature_names, 
                 plot_type='bar', show=False)
plt.title('SHAP Feature Importance - XGBoost', fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../../reports/figures/shap_importance_xgb.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.3 Force Plot - Explicación de Caso Individual (Fraude Detectado)

In [ ]:
# Encuentra un caso de fraude correctamente detectado
y_pred_xgb = model_xgb.predict(X_test_scaled)
fraud_indices = np.where((y_test[:sample_size] == 1) & (y_pred_xgb[:sample_size] == 1))[0]

if len(fraud_indices) > 0:
    fraud_idx = fraud_indices[0]
    
    print(f"\n🔍 Analizando transacción fraudulenta (índice {fraud_idx}):")
    print("="*60)
    for i, fname in enumerate(feature_names):
        print(f"  {fname:20s}: {X_test_sample[fraud_idx, i]:,.2f}")
    print("="*60)
    
    # Force plot
    shap.force_plot(
        explainer_xgb.expected_value,
        shap_values_xgb[fraud_idx],
        X_test_sample[fraud_idx],
        feature_names=feature_names,
        matplotlib=True,
        show=False
    )
    plt.tight_layout()
    plt.savefig('../../reports/figures/shap_force_fraud.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Interpretación del Force Plot:")
    print("  - Base value = predicción promedio del modelo")
    print("  - Features en rojo empujan hacia fraude (1)")
    print("  - Features en azul empujan hacia normal (0)")
    print("  - Longitud de la barra = magnitud del impacto")
else:
    print("⚠️ No se encontraron fraudes en la muestra")

### 3.4 Dependence Plot - Relación entre Features

In [ ]:
# Dependence plot para feature más importante
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Amount (suele ser la más importante)
shap.dependence_plot(
    'amount', shap_values_xgb, X_test_sample,
    feature_names=feature_names,
    ax=axes[0], show=False
)
axes[0].set_title('SHAP Dependence: Amount', fontsize=12, weight='bold')

# newbalanceOrig
shap.dependence_plot(
    'newbalanceOrig', shap_values_xgb, X_test_sample,
    feature_names=feature_names,
    ax=axes[1], show=False
)
axes[1].set_title('SHAP Dependence: newbalanceOrig', fontsize=12, weight='bold')

plt.tight_layout()
plt.savefig('../../reports/figures/shap_dependence_xgb.png', dpi=300)
plt.show()

print("\n📊 Interpretación del Dependence Plot:")
print("  - Muestra cómo cambia el SHAP value con el valor de la feature")
print("  - Color = valor de otra feature (interacción)")
print("  - Permite identificar relaciones no lineales")

## 4. SHAP para Random Forest

In [ ]:
# SHAP para Random Forest
print("📊 Calculando SHAP values para Random Forest...")
explainer_rf = shap.TreeExplainer(model_rf)
shap_values_rf = explainer_rf.shap_values(X_test_sample)

# RF retorna SHAP values para cada clase [clase_0, clase_1]
# Usamos solo la clase 1 (fraude)
if isinstance(shap_values_rf, list):
    shap_values_rf_fraud = shap_values_rf[1]
else:
    shap_values_rf_fraud = shap_values_rf

print(f"✓ SHAP values calculados")
print(f"  Shape: {shap_values_rf_fraud.shape}")

In [ ]:
# Summary plot RF
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values_rf_fraud, X_test_sample,
                 feature_names=feature_names, show=False)
plt.title('SHAP Summary Plot - Random Forest', fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../../reports/figures/shap_summary_rf.png', dpi=300, bbox_inches='tight')
plt.show()

# Bar plot RF
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values_rf_fraud, X_test_sample,
                 feature_names=feature_names,
                 plot_type='bar', show=False)
plt.title('SHAP Feature Importance - Random Forest', fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../../reports/figures/shap_importance_rf.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Comparación de Feature Importance: XGBoost vs Random Forest

In [ ]:
# Calcula importancia promedio por modelo
importance_xgb = np.abs(shap_values_xgb).mean(axis=0)
importance_rf = np.abs(shap_values_rf_fraud).mean(axis=0)

# DataFrame comparativo
comparison_df = pd.DataFrame({
    'Feature': feature_names,
    'XGBoost': importance_xgb,
    'Random Forest': importance_rf
}).sort_values('XGBoost', ascending=False)

print("\n" + "="*60)
print("COMPARACIÓN DE FEATURE IMPORTANCE (SHAP)")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)

# Visualización comparativa
fig, ax = plt.subplots(figsize=(12, 6))
comparison_df.set_index('Feature')[['XGBoost', 'Random Forest']].plot(
    kind='bar', ax=ax, width=0.8
)
ax.set_title('Feature Importance Comparison (SHAP)', fontsize=14, weight='bold')
ax.set_xlabel('Features')
ax.set_ylabel('Mean |SHAP Value|')
ax.legend(title='Modelo')
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../../reports/figures/shap_comparison.png', dpi=300)
plt.show()

## 6. Conclusiones de Explicabilidad

### Hallazgos Clave:

1. **Features más Importantes** (según SHAP):
   - `amount`: Montos de transacción son el predictor #1
   - `newbalanceOrig`: Balance final origen (cuentas vaciadas = fraude)
   - `oldbalanceOrg`: Balance inicial origen
   - `type_encoded`: Tipo de transacción (CASH_OUT/TRANSFER sospechosos)

2. **Patrones Detectados**:
   - Montos altos empujan fuertemente hacia fraude
   - Balances finales cercanos a cero son altamente sospechosos
   - Interacciones no lineales capturadas por los modelos

3. **Consistencia entre Modelos**:
   - XGBoost y Random Forest identifican features similares
   - Alta concordancia = patrones robustos

4. **Cumplimiento Regulatorio**:
   - ✅ SHAP permite explicar cada alerta de fraude
   - ✅ Cumple con transparencia requerida por SARLAFT 2.0
   - ✅ Investigadores pueden justificar decisiones del modelo

### Recomendaciones:

- **Feature Engineering**: Crear más features basadas en `amount` y balances
- **Umbrales**: Usar SHAP para calibrar umbrales de alerta
- **Auditoría**: Force plots individuales para casos sospechosos
- **Monitoreo**: Tracking de feature importance en producción